In [ ]:
from pathlib import Path
import pandas as pd
from lightkurve import search_targetpixelfile

In [ ]:
LOG_FAILED = Path.cwd().parent / "data" / "LOG_FAILED" / "failed_targets.csv"
DATA_FOLDER = Path.cwd().parent / "data"
HOSTS_IDS = Path.cwd().parent / "data" / "HOST_ID" / "filtered_df.csv"

In [ ]:
df = pd.read_csv(HOSTS_IDS)

In [ ]:
for hostname, disposition in zip(df.hostname, df.disposition):
    try:
        tpf = search_targetpixelfile(hostname, author="K2", exptime="long").download(
            quality_bitmask=0
        )
        lc = (
            tpf.to_lightcurve(aperture_mask=tpf.pipeline_mask)
            .remove_nans()
            .remove_outliers()
            .flatten()
        )
        
        lc_df = lc.to_pandas()
        bls_df = lc[["time", "flux", "flux_err"]].copy()
        PATH = DATA_FOLDER / disposition / f"{hostname}.csv"
        bls_df.to_csv(PATH)

    except:
        with open(LOG_FAILED, "a") as file:
            file.write(f"{hostname},{disposition}\n")